In [1]:
import os
import folium
from PIL import Image
import pillow_heif
import piexif
import base64
from io import BytesIO

# Register the HEIF format with Pillow
pillow_heif.register_heif_opener()


In [2]:
def extract_exif_data(image_path):
    """Extract EXIF data from an image."""
    image = Image.open(image_path)
    exif_data = piexif.load(image.info.get("exif", b""))
    return exif_data, image

def get_coordinates_from_exif(exif_data):
    """Extract GPS coordinates from EXIF data."""
    gps_info = exif_data.get("GPS", {})
    
    if not gps_info:
        return None, None
    
    def convert_to_degrees(value):
        """Convert GPS coordinates to degrees in float format."""
        d = float(value[0][0]) / float(value[0][1])
        m = float(value[1][0]) / float(value[1][1])
        s = float(value[2][0]) / float(value[2][1])
        return d + (m / 60.0) + (s / 3600.0)
    
    lat_ref = gps_info[piexif.GPSIFD.GPSLatitudeRef].decode()
    lon_ref = gps_info[piexif.GPSIFD.GPSLongitudeRef].decode()
    
    latitude = convert_to_degrees(gps_info[piexif.GPSIFD.GPSLatitude])
    longitude = convert_to_degrees(gps_info[piexif.GPSIFD.GPSLongitude])
    
    if lat_ref == 'S':
        latitude = -latitude
    if lon_ref == 'W':
        longitude = -longitude
    
    return latitude, longitude

def get_camera_info(exif_data):
    """Extract camera model and date from EXIF data."""
    model = exif_data.get("0th", {}).get(piexif.ImageIFD.Model, b"").decode()
    datetime = exif_data.get("0th", {}).get(piexif.ImageIFD.DateTime, b"").decode()
    return model, datetime

def image_to_base64(image):
    """Convert a PIL image to a base64 string."""
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode()

def create_map_with_images(folder_path='./photos', output_file='map_with_images.html'):
    """Create a folium map with markers showing image coordinates and thumbnails."""
    # List all images in the specified folder
    valid_extensions = ('.heic', '.jpg', '.jpeg', '.png')
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)]

    # Initialize map centered on the first image's coordinates
    coordinates_list = []
    first_lat, first_lon = None, None

    for img_path in image_paths:
        exif_data, image = extract_exif_data(img_path)
        latitude, longitude = get_coordinates_from_exif(exif_data)
        model, datetime = get_camera_info(exif_data)
        
        if latitude is not None and longitude is not None:
            coordinates_list.append((latitude, longitude, image, model, datetime))
            if first_lat is None and first_lon is None:
                first_lat, first_lon = latitude, longitude

    if first_lat is None or first_lon is None:
        print("No valid GPS data found in images.")
        return

    # Create the map
    map_ = folium.Map(location=[first_lat, first_lon], zoom_start=12)

    for latitude, longitude, image, model, datetime in coordinates_list:
        # Resize image for thumbnail
        thumbnail = image.copy()
        thumbnail.thumbnail((200, 200))  # Resize to 200x200 for thumbnail
        thumbnail_base64 = image_to_base64(thumbnail)
        
        # Convert full image to base64
        full_image_base64 = image_to_base64(image)
        full_image_url = f"data:image/jpeg;base64,{full_image_base64}"
        
        # Create HTML for the popup with a link to the full-size image and additional info
        popup_html = f"""
        <div>
            <a href="{full_image_url}" target="_blank"><img src="data:image/jpeg;base64,{thumbnail_base64}"></a><br>
            <strong>Camera:</strong> {model}<br>
            <strong>Date & Time:</strong> {datetime}
        </div>
        """
        
        # Add marker with pop-up
        folium.Marker(
            location=[latitude, longitude],
            popup=folium.Popup(popup_html, max_width=250),
        ).add_to(map_)
    
    # Add Google Hybrid basemap
    google_hybrid = folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Hybrid',
        overlay=False,
        control=True
    ).add_to(map_)

    # Add Google Maps
    google_maps = folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Maps',
        overlay=False,
        control=True
    ).add_to(map_)

    # Add LayerControl to the map
    folium.LayerControl().add_to(map_)

    # Save map to HTML file
    map_.save(output_file)
    print(f"Map with images created: {output_file}")

In [3]:
# Run the function with the folder path './photos'
create_map_with_images('./photos')

Map with images created: map_with_images.html
